# Manipulation Modelling & Execution

The discussed components (KOMO, BotOp, NLP_Solver, RRT) provide basic ingredients for manipulation planning and execution. This tutorial is about how to practically use these in typical manipulation settings.

The first focus is on *manipulation modelling*. While KOMO provides a very powerful abstract framework to define all kinds of constraints, here we discuss what are concrete useful constraints for typical actions, e.g., picking and placing a box, or capsule. The *ManipulationModelling* class is meant to translate between typical actions and the abstract KOMO specification of the corresponding constraints.

The second focus is on the whole pipeline. We follow a basic sense-plan-act pipeline (not yet a fully integrated reactive framework such as SecMPC). To be more detailed, we assume the following basic steps in each loop:
* Perception: Update the configuration model to be in sync with the real world - using perception.
* Discrete decisions (task planning): Decide on discrete next actions, such as which object to pick or place next.
* Waypoint planning: Model the manipulation constraints for the next few actions and solve them to get a plan for the next few waypoints.
* Path planning: Create a fine-grained path/trajectory between waypoints, sometimes justing quick interpolation & optimization, sometimes using full fledge path finding (bi-directional RRT).
* Execution: Sending the path to BotOp for running it on the real system.

We neglect perception and discrete decision making here.

## Manipulation Modelling

We start with discussing manipulation modelling for standard box/cylinder grasping and placing.

In [1]:
import robotic as ry
import numpy as np
import random
import time

# this import the local manipulation.py .. to be integrated in robotic..
import manipulation as manip

A basic configuration with a box and cylinder:

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))

C.addFrame('box') \
    .setPosition([-.25,.1,1.]) \
    .setShape(ry.ST.ssBox, size=[.06,.06,.06,.005]) \
    .setColor([1,.5,0]) \
    .setContact(1)

C.addFrame("capsule") \
    .setShape(ry.ST.capsule, [.2,.02]) \
    .setPosition([.25,.1,1.]) \
    .setColor([1,.5,0]) \
    .setContact(1)

# for convenience, a few definitions:
qHome = C.getJointState()
gripper = "l_gripper";
palm = "l_palm";
box = "box";
table = "table";
boxSize = C.frame(box).getSize()

C.view()

0

Look into the definition of *ManipulationModelling* class! You see that this class provides wrapper methods to setup a komo problem. The following demonstrate the methods provided to model box and cylinder grasping IK problems:

### Box centered top grasp
There are 6 possible orientation of an orthonormal centered box grasp. Have a look at the `grasp_top_box` method!

In [3]:
C.setJointState(qHome)
for orientation in ['xy', 'xz', 'yx', 'yz', 'zx', 'zy']: #loops over the 6 possible grasps
    # setup the manipulation problem
    man = manip.ManipulationModelling(C)
    man.setup_inverse_kinematics()
    man.grasp_top_box(1., gripper, box, orientation)
    
    # solve it
    pose, ret = man.solve()
    
    # check feasibility and display
    if ret.feasible:
        C.setJointState(pose[0])
        C.view(True, f'grasp_top_box with orientation {orientation}\nret: {ret}')
    else:
        print(' -- infeasible')

### Box general grasp
We do not have to grasp a box in the center or orthonormally. We can only specify along which axis the fingers should press, and that they need to be inside a margin of the box sides.  Have a look at the `grasp_box` method! To illustrate the gained degrees of freedom, we also impose a random bias (leading to different solutions in nullspace):

In [4]:
C.setJointState(qHome)
limits = C.getJointLimits()
for orientation in ['x', 'y', 'z']:
    for i in range(10):
        # setup the manipulation problem
        man = manip.ManipulationModelling(C)
        man.setup_inverse_kinematics()
        # ... with random bias in joint space
        qBias = limits[0]+np.random.uniform(qHome.shape)%(limits[1]-limits[0])
        man.bias(1., qBias, 1e0)
        # ... and general, non-centered box grasping
        man.grasp_box(1., gripper, box, palm, orientation, margin=.02)
        
        # solve
        pose, ret = man.solve()
        
        # if feasible, display
        if ret.feasible:
            C.setJointState(pose[0])
            C.view(True, f'grasp_box with orientation {orientation}\nret: {ret}')
        else:
            print('-- infeasible', i, orientation)

-- infeasible 1 y
-- infeasible 8 y


### Cylinder grasp

A cylinder (or capsule) can be grasped by ensuring the finger axis is normal to the cylinder's axis -- have a look at the `grasp_cylinder` method. Again, a demo with random bias to show the variety of grasps modelled that way:

In [5]:
C.setJointState(qHome)
limits = C.getJointLimits()
for i in range(10):
    # setup the manipulation problem
    man = manip.ManipulationModelling(C)
    man.setup_inverse_kinematics()
    qBias = limits[0]+np.random.uniform(qHome.shape)%(limits[1]-limits[0])
    man.bias(1., qBias, 1e0)
    man.grasp_cylinder(1., gripper, 'capsule', palm)
    
    # solve
    pose, ret = man.solve()
    
    # if feasible, display
    if ret.feasible:
        C.setJointState(pose[0])
        C.view(True, f'grasp_cylinder\nret: {ret}')
    else:
        print('-- infeasible', i, orientation)

-- infeasible 0 z


## Sequential Manipulation Modelling

Sequential manipulation modelling is special, as in some phases the manipulated objects move with the manipulator. Internally, komo models this with a *mode switch* (where an object becomes attached to the manipulator with a stable (but optimizable) relative transform).

Using the ManipulationModelling class, the `setup_pick_and_place_waypoints` method creates a two-time-steps komo problem where the relative object-gripper position is constrained to be the same in the 1st and 2st step (as it is parameterized by a shared relative kinematic joint). The `grasp_box` method ensures that the solution *also* fulfils grasp constraints in the first time step; and the `place_box` method ensures that the solution *also* fulfils placement constraints in the second time step. The additional `target_relative_xy_position` is optional, so see placement to explicit xy-positions on the table. Have a look at the definitions of all these methods.

In [25]:
C.setJointState(qHome)

for i in range(10):
    grasp_ori = random.choice(['x', 'y', 'z'])
    place_ori = 'z' #random.choice(['x', 'y', 'z'])
    info = f'pnp {i}, grasp orientation {grasp_ori}, place orientation {place_ori}'
    print('===', info)
    
    # setup manipulation problem
    man = manip.ManipulationModelling(C)
    man.setup_pick_and_place_waypoints(gripper, box)
    man.grasp_box(1., gripper, box, palm, grasp_ori)
    man.place_box(2., box, table, palm, place_ori)
    man.target_relative_xy_position(2., box, table, [(i%5)*.1-.2, .2])
    
    # solve
    q, ret = man.solve()

    # if feasible, display (including 'fake' simulation with kinematic attach)
    if ret.feasible:
        C.setJointState(q[0])
        C.view(True, f'{info}\nwaypoint 0\nret: {ret}')
        C.attach(gripper, box)
        C.setJointState(q[1])
        C.view(True, f'{info}\nwaypoint 1\nret: {ret}')
        C.attach(table, box)
        C.setJointState(qHome)
        C.view(True, 'back home')
    else:
        print(' -- infeasible')

del man
C.frame('box').setPosition([-.25,.1,1.])
C.view()

13

## Path generation

Once solutions to the manipulation keyframes/waypoints are available, the next step is to generate motion between them. We can use sample-based path finding (bi-directional RRT) and/or smooth motion optimization for this.

### Smooth point-to-point motion
The following demonstrates smooth point-to-point motion between box grasps, there the motion is additionally constrains the endeffector to retract and approach:

In [26]:
C.setJointState(qHome)
limits = C.getJointLimits()
verbose = 0

for i in range(20):
    qStart = C.getJointState()
    
    # choose a random grasp orientation
    orientation = random.choice(['x', 'y', 'z'])
    print('===', i, 'orientation', orientation)
    
    # setup the grasp problem
    man = manip.ManipulationModelling(C)
    man.setup_inverse_kinematics(accumulated_collisions=True)
    man.grasp_box(1., gripper, box, palm, orientation)
    
    # solve
    pose, ret = man.solve()
    print('    IK:', ret)
    
    # if feasible, display; otherwise try another grasp
    if ret.feasible:
        if verbose>0:
            C.setJointState(pose[0])
            C.view(True, f'grasp {i} with orientation {orientation}\nret: {ret}')
    else:
        print('  -- infeasible')
        C.setJointState(qStart)
        continue

    # setup the motion problem
    man = manip.ManipulationModelling(C)
    man.setup_point_to_point_motion(qStart, pose[0], accumulated_collisions=True, helpers=[gripper])
    man.endeff_retract([.0, .2], gripper)
    man.endeff_approach([.8, 1.], gripper)
    
    # solve
    path, ret = man.solve()
    print('  path:', ret)

    # if feasible, display trivially (no real execution in BotOp here)
    if ret.feasible:
        for t in range(path.shape[0]):
            C.setJointState(path[t])
            C.view(False, f'grasp {i} with orientation {orientation}, path step {t}\n{ret}')
            time.sleep(.05)
        C.view(verbose>0, f'path done')
    else:
        print('  -- infeasible')
        
del man

=== 0 orientation z
    IK: { time: 0.016926, evals: 38, done: 1, feasible: 1, sos: 0.0547024, f: 0, ineq: 0, eq: 5.42081e-07 }
  path: { time: 0.051555, evals: 33, done: 1, feasible: 1, sos: 2.32512, f: 0, ineq: 8.16764e-07, eq: 4.89434e-05 }
=== 1 orientation y
    IK: { time: 0.033514, evals: 39, done: 1, feasible: 1, sos: 0.00679636, f: 0, ineq: 3.17618e-06, eq: 4.31737e-05 }
  path: { time: 0.035542, evals: 32, done: 1, feasible: 1, sos: 2.06634, f: 0, ineq: 8.45329e-07, eq: 0.000303098 }
=== 2 orientation z
    IK: { time: 0.019448, evals: 41, done: 1, feasible: 1, sos: 0.0547024, f: 0, ineq: 0, eq: 5.26221e-07 }
  path: { time: 0.040584, evals: 20, done: 1, feasible: 1, sos: 2.01013, f: 0, ineq: 9.68448e-07, eq: 0.000912529 }
=== 3 orientation y
    IK: { time: 0.038841, evals: 59, done: 1, feasible: 1, sos: 0.00679636, f: 0, ineq: 3.11412e-06, eq: 4.40807e-05 }
  path: { time: 0.037002, evals: 35, done: 1, feasible: 1, sos: 2.06634, f: 0, ineq: 8.4471e-07, eq: 0.000303112 }
===

## Integrated Example

Let's start with an integrated example, where the robot endlessly loops through picking and placing a box on a table.

In [27]:
exit()

In [1]:
import robotic as ry
import manipulation as manip
from importlib import reload
import time
import random

We define a basic configuration with box on the table:

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))

C.addFrame("box", "table") \
    .setJoint(ry.JT.rigid) \
    .setShape(ry.ST.ssBox, [.06,.15,.06,.005]) \
    .setRelativePosition([-.0,.3-.055,.095]) \
    .setMass(.1)

# for convenience, a few definitions:
qHome = C.getJointState()
gripper = "l_gripper";
palm = "l_palm";
box = "box";
table = "table";
boxSize = C.frame(box).getSize()

C.view()

0

Here is the full loop, to be explained below:

In [3]:
C.setJointState(qHome)
C.view_raise()

for l in range(10):
        qStart = C.getJointState()

        graspDirection = random.choice(['x', 'z']) #'y' not possible: box too large
        placeDirection = random.choice(['x', 'y', 'z', 'xNeg', 'yNeg', 'zNeg'])
        info = f'placement {l}: grasp {graspDirection} place {placeDirection}'
        print('===', info)

        M = manip.ManipulationModelling(C)
        M.setup_pick_and_place_waypoints(gripper, box, homing_scale=1e-1)
        M.grasp_box(1., gripper, box, palm, graspDirection)
        M.no_collision([1.], palm, table)
        M.place_box(2., box, table, palm, placeDirection)
        M.target_relative_xy_position(2., box, table, [.2, .3])
        ways, ret = M.solve(verbose=0)

#         M.komo.view(True, f'waypoint planning result\n{info}\nfeasible: {ret.feasible}')

        if not ret.feasible:
                print('  -- waypoints not feasible')
                continue

        M = manip.ManipulationModelling(C)
        M.setup_point_to_point_motion(qStart, ways[0], helpers=[gripper])
        M.no_collision([.3,.7], palm, box, margin=.05)
        M.endeff_retract([.0, .2], gripper)
        M.endeff_approach([.8, 1.], gripper)
        path, ret = M.solve(verbose=0)

        if not ret.feasible:
            print('  -- pick not feasible')
            continue
        else:
            # bot.move(path, [2.])
            # bot.wait(C)
            for t in range(path.shape[0]):
                C.setJointState(path[t])
                C.view(False, f'pick step {t}\n{info}')
                time.sleep(.1)

        C.attach(gripper, box)

        M = manip.ManipulationModelling(C)
        M.setup_point_to_point_motion(ways[0], ways[1])
        M.no_collision([], table, box)
        path, ret = M.solve(verbose=0)

        if not ret.feasible:
            print('  -- place not feasible')
        else:
#             bot.move(path, [2.])
#             bot.wait(C)
            for t in range(path.shape[0]):
                C.setJointState(path[t])
                C.view(False, f'place step {t}\n{info}')
                time.sleep(.1)

        C.attach(table, box)

del M

=== placement 0: grasp x place zNeg
  -- waypoints not feasible
=== placement 1: grasp x place yNeg
=== placement 2: grasp z place xNeg
=== placement 3: grasp z place y
=== placement 4: grasp z place z
  -- waypoints not feasible
=== placement 5: grasp x place yNeg
  -- waypoints not feasible
=== placement 6: grasp z place x
=== placement 7: grasp z place xNeg
  -- waypoints not feasible
=== placement 8: grasp z place y
=== placement 9: grasp z place y
=== placement 10: grasp z place y
=== placement 11: grasp x place yNeg
  -- waypoints not feasible
=== placement 12: grasp z place z
  -- waypoints not feasible
=== placement 13: grasp z place zNeg
  -- waypoints not feasible
=== placement 14: grasp z place x
=== placement 15: grasp x place z
  -- waypoints not feasible
=== placement 16: grasp z place x
=== placement 17: grasp z place z
  -- waypoints not feasible
=== placement 18: grasp x place x
  -- waypoints not feasible
=== placement 19: grasp x place xNeg
  -- waypoints not feasibl

## TODOS:
* Proper execution: BotOp instead of display with C
* RRTs
* additional planar motion constraint for in-plane manipulation
* more typical manipulation constraints: camera_look_at, push_straight, 

In [4]:
del C